In [1]:
import matplotlib.pyplot as plt
import numpy as np
from numpy import linalg as LA
from numpy.linalg import matrix_power as MP

import random
import scipy
from scipy.spatial.distance import cdist
from scipy.io import savemat, loadmat

import networkx as nx
import sys

import tensorflow as tf
from tensorflow.math import scalar_mul as c_mul
from keras.callbacks import EarlyStopping

print(f"TF Version: {tf.__version__}")

TF Version: 2.14.0


In [2]:
COBRE_dataset = loadmat("./Dataset/COBRE_ROIdata_for_132_and_32_ROIs.mat")
COBRE_dataset_132 = COBRE_dataset['COBRE_ROIdata_132']
COBRE_dataset_32 = COBRE_dataset['COBRE_ROIdata_32']
COBRE_labels = [0] * 74 + [1] * 71 # 0 - Control, 1 - Patient

X_train_data = COBRE_dataset_32
y_train_data = COBRE_labels

N, T, num_data = X_train_data.shape
print(N, T, num_data)

32 150 145


In [3]:
def generate_delta(T):
    diff_matrix = np.zeros([T, T - 1]) - np.eye(T, T - 1)
    for i in range(1, T):
        diff_matrix[i, i - 1] = 1
    return tf.convert_to_tensor(diff_matrix, dtype = "float32")

def adj_to_laplacian(AdjL):
    tmp = np.array(AdjL, dtype = "float32")
    return np.diag(np.sum(tmp, axis = 0)) - tmp

def initial_graph(X_training, N, top_values = 6):
    distance_matrix = cdist(X_training, X_training, metric = 'euclidean')
    
    # top_values = 6
    sorted_indices = distance_matrix.argsort(1)
    TAdj = np.zeros([N, N])
    
    for i in range(N):
        for j in range(1, top_values):
            k = sorted_indices[i][j]
            TAdj[i][k] = 1
    
    TAdj = (TAdj + TAdj.T) / 2
    TAdj = np.where(TAdj > 0.1, 1, 0)
    
    Laplacian_init = np.diag(np.sum(TAdj, 1)) - TAdj
    
    return Laplacian_init

def label_print(idx, length):
    l = len(str(idx))
    if l < length:
        return "0" * (length - l) + str(idx)
    else:
        return str(idx)

In [4]:
d = generate_delta(T)
print(d)

Mul = np.eye(N) - np.ones([N, N])
Mul = tf.convert_to_tensor(Mul, dtype = "float32")

tf.Tensor(
[[-1.  0.  0. ...  0.  0.  0.]
 [ 1. -1.  0. ...  0.  0.  0.]
 [ 0.  1. -1. ...  0.  0.  0.]
 ...
 [ 0.  0.  0. ...  1. -1.  0.]
 [ 0.  0.  0. ...  0.  1. -1.]
 [ 0.  0.  0. ...  0.  0.  1.]], shape=(150, 149), dtype=float32)


In [5]:
coeffs = 0
psi_cost = 0
Laplacian = 0
A_transform_cost = 0

model_pred = []
loss_model = []
learned_graph = []
mse_known_list = []
mse_unknown_list = []

In [6]:
coeffs_list = np.load("./Outputs/COBRE_cost_func_coeffs.npy")
print(coeffs_list.shape)

(145, 4)


In [7]:
# A - Target, B - Predicted
def costfunc(A, B):
    global coeffs
    global X_train
    global psi_cost
    global Laplacian
    global A_transform_cost

    B_tmp = tf.squeeze(B)
    data_fidelity = (tf.norm(tf.multiply(psi_cost, (X_train - B))) ** 2) * coeffs[0]
    smoothness = tf.linalg.trace(tf.transpose(B_tmp) @ Laplacian @ B_tmp @ A_transform_cost) * coeffs[1]
    L_cost = (tf.norm(Laplacian) ** 2) * coeffs[2]
    Z_cost = (tf.norm(A_transform_cost) ** 2) * coeffs[3]
    
    tmp = (smoothness) + (data_fidelity) + (L_cost) + (Z_cost)
    
    global loss_model
    loss_model.append(tmp)
    
    return tmp
    
def costfunc1(A, B):
    global X_train
    global psi_cost
    
    data_fidelity = (tf.norm(tf.multiply(psi_cost, (X_train - B))) ** 2)
    return (data_fidelity)

def costfunc2(A, B):
    global Laplacian
    global A_transform_cost
    
    B_tmp = tf.squeeze(B)
    smoothness = tf.linalg.trace(tf.transpose(B_tmp) @ Laplacian @ B_tmp @ A_transform_cost)
    
    return (smoothness)

def costfunc3(A, B):
    global Laplacian
    
    L_cost = (tf.norm(Laplacian) ** 2)
    
    return (L_cost)

def costfunc4(A, B):
    global A_transform_cost
    
    Z_cost = (tf.norm(A_transform_cost) ** 2)
    
    return (Z_cost)

In [8]:
def mse_unknown(A, B):
    
    global psi_cost
    psi = psi_cost
    
    psid = tf.convert_to_tensor(np.ones([N, T], dtype = float) - psi, dtype = "float32")
    mse_unk = (LA.norm(np.multiply(psid, (X_train - B))) ** 2) / np.sum(psid.numpy().flatten())
    
    global mse_unknown_list
    mse_unknown_list.append(mse_unk)
    
    return mse_unk

def mse_known(A, B):
    
    global psi_cost
    psi = psi_cost
    
    mse_kn = (LA.norm(np.multiply(psi, (X_train - B))) ** 2) / np.sum(psi.numpy().flatten())
    
    global model_pred
    model_pred.append([A.numpy(), B.numpy()])
    
    global mse_known_list
    mse_known_list.append(mse_kn)
    
    return mse_kn

In [9]:
callb = EarlyStopping(monitor = 'loss', mode = 'min', min_delta = 0.0001, verbose = 1, patience = 10)

lr_decay_factor = 1.2
def scheduler(epoch, lr):
    if epoch < 3:
        return lr
    else:
        return lr / lr_decay_factor

lr_scheduler = tf.keras.callbacks.LearningRateScheduler(scheduler)

In [10]:
class ProposedModel(tf.keras.Model):
    
    def __init__(self, L_init, reg_init, loops_init, d_init, p_init, alpha_init, beta_init, threshold_in, **kwargs):
        super(ProposedModel, self).__init__(**kwargs)
        
        self.p1_init = tf.constant(p_init[0], shape = (1, 1), dtype = "float32")
        self.p1 = tf.Variable(initial_value = self.p1_init, trainable = True, name = "p1")
        
        self.p2_init = tf.constant(p_init[1], shape = (1, 1), dtype = "float32")
        self.p2 = tf.Variable(initial_value = self.p2_init, trainable = True, name = "p2")
        
        self.L_in = L_init
        self.preg = reg_init
        self.loops = loops_init
        self.d_int = d_init
        self.alp = alpha_init
        self.bet = beta_init
        self.threshold_init = threshold_in
        
    def data_inpainting(self, psi, Y, Lk, AAT):
        loop_b, reg = self.loops[1], self.preg

        Xk = tf.zeros_like(Y)
        Zk = -(tf.multiply(psi, Xk) - Y + (2 * reg * (Lk @ Xk @ AAT)))
        
        for i in range(loop_b):
            fdx_xk = tf.multiply(psi, Xk) - Y + (2 * reg * (Lk @ Xk @ AAT))

            # row_print = 4
            # tmp = list(zip((tf.multiply(psi, Xk) - Y)[:, row_print].numpy(), (2 * reg * (Lk @ Xk @ AAT)[:, row_print].numpy())))
            # print("\n\nStart here")
            # for k in tmp:
            #     print(k)
            
            fdx_zk = tf.multiply(psi, Zk) - Y + (2 * reg * (Lk @ Zk @ AAT))
            
            tau = tf.linalg.trace(tf.transpose(fdx_xk) @ Zk) / tf.linalg.trace(tf.transpose((Y + fdx_zk)) @ Zk)
            
            Xk_1 = Xk - (tau * Zk)

            fdx_xk_1 = tf.multiply(psi, Xk_1) - Y + (2 * reg * (Lk @ Xk_1 @ AAT))
            gamma = (tf.norm(fdx_xk_1) ** 2) / (tf.norm(fdx_xk) ** 2)

            Zk_1 = (gamma * Zk) - fdx_xk_1

            Xk = Xk_1
            Zk = Zk_1
        # print("\n\nEnds here\n\n")
        return Xk_1 
    
    def graph_learning(self, X, Lk, AAT):
        loop_c, alpha, beta = self.loops[2], self.alp, self.bet
        
        # Computing the smoothness term
        X_AAT_XT = tf.matmul(X, tf.matmul(AAT, tf.transpose(X)))
        
        for i in range(loop_c):

            # Extracting the adjacency matrix from the Laplacian
            adj_i = tf.multiply(Mul, Lk)

            # Computing the gradient with respect to laplacian
            fdL_Lk = X_AAT_XT + (beta * Lk)
            
            # row_print = 4
            # print("For beta")
            # tmp = list(zip(X_AAT_XT[row_print, :].numpy(), np.array(beta * Lk)[row_print, :]))
            # for k in tmp:
            #     print(k)
            
            # Extracting the adjacency matrix from the gradient
            adj_grad = tf.multiply(Mul, fdL_Lk)
            
            # Computing the new adjacency
            # tmp = list(zip(adj_i[row_print, :].numpy(), (alpha * adj_grad)[row_print, :].numpy()))
            # if i == (loop_c-1):
            #     print("\nFor alpha")
            #     for k in tmp:
            #         print(k)
            #     print("\nStart here")
            
            adj_i1 = adj_i - (alpha * adj_grad)
            
            # Avoiding any negative values
            adj_i1 = tf.nn.relu(adj_i1)

            # Computing the new laplacian
            Lk_1 = tf.linalg.diag(tf.math.reduce_sum(adj_i1, 1)) - adj_i1
            
            # Updating the laplacian
            Lk = Lk_1
        # print("\n\nEnds here\n\n")
        return Lk
    

    def call(self, y):

        psi = y[0,:,T:]
        global psi_cost
        psi_cost = psi

        psi = tf.convert_to_tensor(psi, dtype = "float32")
        
        X_in = y[0,:,:T]
        X_in = psi * tf.convert_to_tensor(X_in, dtype = "float32")
        Y = X_in
        
        d = self.d_int
        LT = tf.matmul(d, tf.transpose(d))
        LT = tf.convert_to_tensor(LT, dtype = "float32")
        
        A_transform = tf.eye(T) + (self.p1 * MP(LT, 1)) + (self.p2 * MP(LT, 2))
        AAT = A_transform
        
        global A_transform_cost
        A_transform_cost = AAT
        
        loop_a = self.loops[0]
        threshold = self.threshold_init
        Lk = self.L_in
        
        for i in range(loop_a):
            
            Xk_1 = self.data_inpainting(psi, Y, Lk, AAT)
            
            Xk_1_gt = X_in + tf.multiply((tf.ones(psi.shape) - psi), Xk_1)
            
            Lk_1 = self.graph_learning(Xk_1_gt, Lk, AAT)
            
            Adj = tf.linalg.diag(tf.linalg.diag_part(Lk_1)) - Lk_1 # Gives adjacency
            
            n = tf.math.count_nonzero(Adj, dtype="float32")
            mean = tf.math.reduce_sum(Adj) / n
            Adj = tf.math.subtract(Adj, (threshold * mean)) # Ensures the sparsity of learned graph
            
            Adj = tf.nn.relu(Adj)
            # Adj = tf.math.sign(Adj)
            
            Lk_1 = tf.linalg.diag(tf.math.reduce_sum(Adj, 1)) - Adj
            
            Lk = Lk_1
        
            global Laplacian
            Laplacian = Lk
        
        Xk_1 = self.data_inpainting(psi, Y, Lk, AAT)
        
        global learned_graph
        learned_graph.append(Lk.numpy())
        
        return tf.reshape(Xk_1, [1, N, T])

In [11]:
loop_a = 5
loop_b = 10 # Data Inpainting
loop_c = 10 # Graph Learning

reg_init = 1.0e-2

alpha_init = 1.0e-4
beta_init = 1.0e-2

threshold_init = 0.3

no_of_psi = 20
sensing_ratio = 0.1

p_init = [30.0, 60.0]
lr_decay_factor = 1.2

In [12]:
all_vars = []
for dat in range(num_data):
    print(f"Datapoint: {dat + 1}")
    model_pred = []
    loss_model = []
    learned_graph = []

    X_train = X_train_data[:, :, dat]
    Lap_init = initial_graph(X_train, N, 6)
    
    all_psi = []

    M = int(sensing_ratio * T)
    X_train_missing = np.zeros([no_of_psi, N, T])
    X_train_concatenated = np.zeros([no_of_psi, N, 2 * T])

    for i in range(no_of_psi):
        psi_k = np.array([0] * (N * M) + [1] * (N * (T - M)))
        np.random.shuffle(psi_k)
        psi_k = psi_k.reshape([N, T])
        all_psi.append(psi_k)
        X_train_missing[i, : , : ] = X_train * psi_k
        X_train_concatenated[i, :, :] = np.concatenate((X_train_missing[i], psi_k), axis = 1)
    
    coeffs = coeffs_list[dat, :]

    sgd_optimizer = tf.keras.optimizers.SGD(learning_rate = 0.05, momentum = 0.0)
    
    our_model = ProposedModel(Lap_init, reg_init, [loop_a, loop_b, loop_c], 
                              d, p_init, alpha_init, beta_init, threshold_init, name = 'our')
    our_model.compile(optimizer = sgd_optimizer, loss = costfunc,
                      metrics = [mse_known, mse_unknown], run_eagerly=True)
    
    our_model.fit(X_train_concatenated, X_train_missing, epochs = 15, 
                  callbacks = [lr_scheduler], batch_size = 1)
    
    var = our_model.variables
    var_lst = []
    for v in var:
        var_lst.append(v.numpy()[0, 0])
    print(f"\nFilter Coefficients:", var_lst)
    all_vars.append(var_lst)

    graph = learned_graph[-1]
    print(f"\nLearned Graph Shape:\n{graph.shape}")

    np.save(f"./Outputs/Graphs/COBRE/{label_print(dat, 3)}_{y_train_data[dat]}.npy", graph)

Datapoint: 1


Epoch 1/15
20/20 [==============================] - 14s 661ms/step - loss: 159.7342 - mse_known: 0.3631 - mse_unknown: 0.3882 - lr: 0.0500
Epoch 2/15
20/20 [==============================] - 12s 618ms/step - loss: 141.8947 - mse_known: 0.3546 - mse_unknown: 0.3829 - lr: 0.0500
Epoch 3/15
20/20 [==============================] - 13s 644ms/step - loss: 126.1428 - mse_known: 0.3454 - mse_unknown: 0.3773 - lr: 0.0500
Epoch 4/15
20/20 [==============================] - 13s 679ms/step - loss: 113.2068 - mse_known: 0.3363 - mse_unknown: 0.3715 - lr: 0.0417
Epoch 5/15
20/20 [==============================] - 13s 667ms/step - loss: 103.4942 - mse_known: 0.3282 - mse_unknown: 0.3664 - lr: 0.0347
Epoch 6/15
20/20 [==============================] - 13s 672ms/step - loss: 96.0678 - mse_known: 0.3213 - mse_unknown: 0.3620 - lr: 0.0289
Epoch 7/15
20/20 [==============================] - 13s 663ms/step - loss: 90.2957 - mse_known: 0.3152 - mse_unknown: 0.3581 - lr: 0.0241
Epoch 8/15
20/20 [===========


Filter Coefficients: [18.252048, 35.813023]

Learned Graph Shape:
(32, 32)
Datapoint: 2
Epoch 1/15
20/20 [==============================] - 14s 692ms/step - loss: 102.1369 - mse_known: 0.8106 - mse_unknown: 0.8445 - lr: 0.0500
Epoch 2/15
20/20 [==============================] - 13s 659ms/step - loss: 93.6075 - mse_known: 0.8058 - mse_unknown: 0.8422 - lr: 0.0500
Epoch 3/15
20/20 [==============================] - 13s 661ms/step - loss: 85.8308 - mse_known: 0.8007 - mse_unknown: 0.8397 - lr: 0.0500
Epoch 4/15
20/20 [==============================] - 13s 650ms/step - loss: 79.2707 - mse_known: 0.7955 - mse_unknown: 0.8372 - lr: 0.0417
Epoch 5/15
20/20 [==============================] - 13s 657ms/step - loss: 74.2183 - mse_known: 0.7911 - mse_unknown: 0.8350 - lr: 0.0347
Epoch 6/15
20/20 [==============================] - 12s 626ms/step - loss: 70.2762 - mse_known: 0.7871 - mse_unknown: 0.8331 - lr: 0.0289
Epoch 7/15
20/20 [==============================] - 13s 628ms/step - loss: 67.1655


Filter Coefficients: [23.876104, 41.7502]

Learned Graph Shape:
(32, 32)
Datapoint: 3
Epoch 1/15
20/20 [==============================] - 13s 650ms/step - loss: 109.0838 - mse_known: 0.4956 - mse_unknown: 0.5469 - lr: 0.0500
Epoch 2/15
20/20 [==============================] - 13s 630ms/step - loss: 102.0110 - mse_known: 0.4915 - mse_unknown: 0.5449 - lr: 0.0500
Epoch 3/15
20/20 [==============================] - 12s 618ms/step - loss: 95.4683 - mse_known: 0.4871 - mse_unknown: 0.5428 - lr: 0.0500
Epoch 4/15
20/20 [==============================] - 13s 669ms/step - loss: 89.8700 - mse_known: 0.4827 - mse_unknown: 0.5406 - lr: 0.0417
Epoch 5/15
20/20 [==============================] - 13s 663ms/step - loss: 85.4963 - mse_known: 0.4790 - mse_unknown: 0.5388 - lr: 0.0347
Epoch 6/15
20/20 [==============================] - 13s 654ms/step - loss: 82.0358 - mse_known: 0.4756 - mse_unknown: 0.5371 - lr: 0.0289
Epoch 7/15
20/20 [==============================] - 13s 649ms/step - loss: 79.2813 


Filter Coefficients: [24.244999, 43.032272]

Learned Graph Shape:
(32, 32)
Datapoint: 4
Epoch 1/15
20/20 [==============================] - 13s 647ms/step - loss: 78.8913 - mse_known: 0.7477 - mse_unknown: 0.7273 - lr: 0.0500
Epoch 2/15
20/20 [==============================] - 14s 682ms/step - loss: 73.8715 - mse_known: 0.7444 - mse_unknown: 0.7256 - lr: 0.0500
Epoch 3/15
20/20 [==============================] - 13s 682ms/step - loss: 69.1876 - mse_known: 0.7409 - mse_unknown: 0.7238 - lr: 0.0500
Epoch 4/15
20/20 [==============================] - 13s 662ms/step - loss: 65.1603 - mse_known: 0.7375 - mse_unknown: 0.7221 - lr: 0.0417
Epoch 5/15
20/20 [==============================] - 13s 638ms/step - loss: 61.9997 - mse_known: 0.7346 - mse_unknown: 0.7205 - lr: 0.0347
Epoch 6/15
20/20 [==============================] - 13s 646ms/step - loss: 59.4930 - mse_known: 0.7321 - mse_unknown: 0.7192 - lr: 0.0289
Epoch 7/15
20/20 [==============================] - 12s 623ms/step - loss: 57.4896 

...


Filter Coefficients: [18.154913, 30.852512]

Learned Graph Shape:
(32, 32)
Datapoint: 141
Epoch 1/15
20/20 [==============================] - 14s 677ms/step - loss: 78.7893 - mse_known: 1.3907 - mse_unknown: 1.4054 - lr: 0.0500
Epoch 2/15
20/20 [==============================] - 13s 648ms/step - loss: 74.2160 - mse_known: 1.3863 - mse_unknown: 1.4028 - lr: 0.0500
Epoch 3/15
20/20 [==============================] - 13s 669ms/step - loss: 69.9376 - mse_known: 1.3819 - mse_unknown: 1.4002 - lr: 0.0500
Epoch 4/15
20/20 [==============================] - 13s 679ms/step - loss: 66.2433 - mse_known: 1.3774 - mse_unknown: 1.3976 - lr: 0.0417
Epoch 5/15
20/20 [==============================] - 13s 671ms/step - loss: 63.3347 - mse_known: 1.3736 - mse_unknown: 1.3954 - lr: 0.0347
Epoch 6/15
20/20 [==============================] - 14s 677ms/step - loss: 61.0287 - mse_known: 1.3703 - mse_unknown: 1.3935 - lr: 0.0289
Epoch 7/15
20/20 [==============================] - 13s 675ms/step - loss: 59.180


Filter Coefficients: [25.19929, 46.262856]

Learned Graph Shape:
(32, 32)
Datapoint: 142
Epoch 1/15
20/20 [==============================] - 14s 676ms/step - loss: 205.0881 - mse_known: 0.5806 - mse_unknown: 0.6537 - lr: 0.0500
Epoch 2/15
20/20 [==============================] - 14s 681ms/step - loss: 176.0558 - mse_known: 0.5659 - mse_unknown: 0.6438 - lr: 0.0500
Epoch 3/15
20/20 [==============================] - 13s 658ms/step - loss: 151.3809 - mse_known: 0.5495 - mse_unknown: 0.6327 - lr: 0.0500
Epoch 4/15
20/20 [==============================] - 14s 678ms/step - loss: 131.8954 - mse_known: 0.5327 - mse_unknown: 0.6211 - lr: 0.0417
Epoch 5/15
20/20 [==============================] - 13s 663ms/step - loss: 117.7152 - mse_known: 0.5175 - mse_unknown: 0.6105 - lr: 0.0347
Epoch 6/15
20/20 [==============================] - 13s 666ms/step - loss: 107.1499 - mse_known: 0.5042 - mse_unknown: 0.6012 - lr: 0.0289
Epoch 7/15
20/20 [==============================] - 13s 666ms/step - loss: 9


Filter Coefficients: [15.318382, 30.556236]

Learned Graph Shape:
(32, 32)
Datapoint: 143
Epoch 1/15
20/20 [==============================] - 13s 664ms/step - loss: 188.0490 - mse_known: 0.5887 - mse_unknown: 0.6400 - lr: 0.0500
Epoch 2/15
20/20 [==============================] - 14s 688ms/step - loss: 164.3761 - mse_known: 0.5750 - mse_unknown: 0.6318 - lr: 0.0500
Epoch 3/15
20/20 [==============================] - 14s 682ms/step - loss: 143.8926 - mse_known: 0.5599 - mse_unknown: 0.6229 - lr: 0.0500
Epoch 4/15
20/20 [==============================] - 13s 664ms/step - loss: 127.4256 - mse_known: 0.5447 - mse_unknown: 0.6136 - lr: 0.0417
Epoch 5/15
20/20 [==============================] - 14s 684ms/step - loss: 115.2373 - mse_known: 0.5306 - mse_unknown: 0.6050 - lr: 0.0347
Epoch 6/15
20/20 [==============================] - 14s 681ms/step - loss: 106.0370 - mse_known: 0.5182 - mse_unknown: 0.5973 - lr: 0.0289
Epoch 7/15
20/20 [==============================] - 14s 683ms/step - loss: 


Filter Coefficients: [19.001421, 31.769922]

Learned Graph Shape:
(32, 32)
Datapoint: 144
Epoch 1/15
20/20 [==============================] - 14s 695ms/step - loss: 91.1331 - mse_known: 0.3500 - mse_unknown: 0.3556 - lr: 0.0500
Epoch 2/15
20/20 [==============================] - 14s 679ms/step - loss: 85.7945 - mse_known: 0.3479 - mse_unknown: 0.3545 - lr: 0.0500
Epoch 3/15
20/20 [==============================] - 14s 679ms/step - loss: 80.8134 - mse_known: 0.3457 - mse_unknown: 0.3535 - lr: 0.0500
Epoch 4/15
20/20 [==============================] - 14s 677ms/step - loss: 76.5150 - mse_known: 0.3436 - mse_unknown: 0.3525 - lr: 0.0417
Epoch 5/15
20/20 [==============================] - 14s 684ms/step - loss: 73.1316 - mse_known: 0.3418 - mse_unknown: 0.3516 - lr: 0.0347
Epoch 6/15
20/20 [==============================] - 13s 660ms/step - loss: 70.4470 - mse_known: 0.3402 - mse_unknown: 0.3508 - lr: 0.0289
Epoch 7/15
20/20 [==============================] - 14s 694ms/step - loss: 68.296


Filter Coefficients: [25.568335, 44.93176]

Learned Graph Shape:
(32, 32)
Datapoint: 145
Epoch 1/15
20/20 [==============================] - 13s 680ms/step - loss: 123.8218 - mse_known: 0.3696 - mse_unknown: 0.4034 - lr: 0.0500
Epoch 2/15
20/20 [==============================] - 13s 659ms/step - loss: 113.5489 - mse_known: 0.3630 - mse_unknown: 0.3990 - lr: 0.0500
Epoch 3/15
20/20 [==============================] - 14s 672ms/step - loss: 104.1623 - mse_known: 0.3560 - mse_unknown: 0.3943 - lr: 0.0500
Epoch 4/15
20/20 [==============================] - 13s 676ms/step - loss: 96.2587 - mse_known: 0.3492 - mse_unknown: 0.3897 - lr: 0.0417
Epoch 5/15
20/20 [==============================] - 13s 665ms/step - loss: 90.1530 - mse_known: 0.3434 - mse_unknown: 0.3857 - lr: 0.0347
Epoch 6/15
20/20 [==============================] - 14s 686ms/step - loss: 85.3810 - mse_known: 0.3383 - mse_unknown: 0.3823 - lr: 0.0289
Epoch 7/15
20/20 [==============================] - 14s 677ms/step - loss: 81.6

In [13]:
np.save("./Outputs/COBRE_mse_known.npy", mse_known_list)
np.save("./Outputs/COBRE_mse_unknown.npy", mse_unknown_list)

In [14]:
# re_run_idx = [12, 24, 26, 76, 80, 82, 84, 85, 86, 89, 98, 105, 107, 114, 118, 120, 121, 124, 128, 130]

# all_vars = []
# for dat in re_run_idx:
#     dat = dat - 1
#     print(f"Datapoint: {dat + 1}")
#     model_pred = []
#     loss_model = []
#     learned_graph = []

#     X_train = X_train_data[:, :, dat]
#     Lap_init = initial_graph(X_train, N, 6)
    
#     all_psi = []

#     M = int(sensing_ratio * T)
#     X_train_missing = np.zeros([no_of_psi, N, T])
#     X_train_concatenated = np.zeros([no_of_psi, N, 2 * T])

#     for i in range(no_of_psi):
#         psi_k = np.array([0] * (N * M) + [1] * (N * (T - M)))
#         np.random.shuffle(psi_k)
#         psi_k = psi_k.reshape([N, T])
#         all_psi.append(psi_k)
#         X_train_missing[i, : , : ] = X_train * psi_k
#         X_train_concatenated[i, :, :] = np.concatenate((X_train_missing[i], psi_k), axis = 1)
    
#     coeffs = coeffs_list[dat, :]

#     sgd_optimizer = tf.keras.optimizers.SGD(learning_rate = 0.0001, momentum = 0.0)
    
#     our_model = ProposedModel(Lap_init, reg_init, [loop_a, loop_b, loop_c], 
#                               d, p_init, alpha_init, beta_init, threshold_init, name = 'our')
#     our_model.compile(optimizer = sgd_optimizer, loss = costfunc,
#                       metrics = [mse_known, mse_unknown], run_eagerly=True)
    
#     our_model.fit(X_train_concatenated, X_train_missing, epochs = 15, 
#                   callbacks = [lr_scheduler], batch_size = 1)
    
#     var = our_model.variables
#     var_lst = []
#     for v in var:
#         var_lst.append(v.numpy()[0, 0])
#     print(f"\nFilter Coefficients:", var_lst)
#     all_vars.append(var_lst)

#     graph = learned_graph[-1]
#     print(f"\nLearned Graph Shape:\n{graph.shape}")

#     np.save(f"./Outputs/Graphs/COBRE/{label_print(dat, 3)}_{y_train_data[dat]}.npy", graph)